In [1]:
import pandas as pd
from pathlib import Path

cleaned_path = Path("C:/Users/bisar/My fitbit dashboard project/Cleaned")

metrics = [
    "calories", "distance", "lightly_active_minutes", "moderately_active_minutes",
    "sedentary_minutes", "steps", "very_active_minutes"
]

cleaned_data = {}

for metric in metrics:
    cleaned_data[metric] = pd.read_parquet(cleaned_path / f"{metric}.parquet")


In [2]:
cleaned_data

{'calories':                   dateTime  value               source_file    metric
 0      2025-05-04 00:00:00   0.99  calories-2025-05-04.json  calories
 1      2025-05-04 00:01:00   0.99  calories-2025-05-04.json  calories
 2      2025-05-04 00:02:00   0.99  calories-2025-05-04.json  calories
 3      2025-05-04 00:03:00   0.99  calories-2025-05-04.json  calories
 4      2025-05-04 00:04:00   0.99  calories-2025-05-04.json  calories
 ...                    ...    ...                       ...       ...
 351177 2026-01-02 20:57:00   0.99  calories-2025-12-30.json  calories
 351178 2026-01-02 20:58:00   0.99  calories-2025-12-30.json  calories
 351179 2026-01-02 20:59:00   0.99  calories-2025-12-30.json  calories
 351180 2026-01-02 21:00:00   0.99  calories-2025-12-30.json  calories
 351181 2026-01-02 21:01:00   0.99  calories-2025-12-30.json  calories
 
 [351182 rows x 4 columns],
 'distance':                  dateTime  value               source_file    metric
 0     2025-05-05 18:06:

***Merging data into one table***

In [3]:
df_integrated = cleaned_data["calories"].copy()
df_integrated = df_integrated.rename(columns={"value": "calories"})

In [4]:
for metric in ["distance", "steps", "lightly_active_minutes", "moderately_active_minutes", "very_active_minutes", "sedentary_minutes"]:
    temp_df = cleaned_data[metric].copy()
    temp_df = temp_df.rename(columns={"value": metric})

    df_integrated = df_integrated.merge(temp_df[["dateTime", metric]], on="dateTime", how="left")

In [5]:
df_integrated.head()

,dateTime,calories,source_file,metric,distance,steps,lightly_active_minutes,moderately_active_minutes,very_active_minutes,sedentary_minutes
0,2025-05-04 00:00:00,0.99,calories-2025-05-04.json,calories,NaN,NaN,0.0,0.0,0.0,1440.0
1,2025-05-04 00:01:00,0.99,calories-2025-05-04.json,calories,NaN,NaN,NaN,NaN,NaN,NaN
2,2025-05-04 00:02:00,0.99,calories-2025-05-04.json,calories,NaN,NaN,NaN,NaN,NaN,NaN
3,2025-05-04 00:03:00,0.99,calories-2025-05-04.json,calories,NaN,NaN,NaN,NaN,NaN,NaN
4,2025-05-04 00:04:00,0.99,calories-2025-05-04.json,calories,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
df_integrated.to_parquet("C:/Users/bisar/My fitbit dashboard project/Cleaned/df_integrated.parquet", index=False)
df_integrated.to_csv("C:/Users/bisar/My fitbit dashboard project/Cleaned/df_integrated.csv", index=False)

**Data Aggregation per day**

In [7]:
# Make a copy of integrated data
df_daily = df_integrated.copy()

# Extract date only
df_daily['date'] = df_daily['dateTime'].dt.date

# Define aggregation rules for each metric
agg_rules = {
    'calories': 'sum',
    'distance': 'sum',
    'steps': 'sum',
    'lightly_active_minutes': 'sum',
    'moderately_active_minutes': 'sum',
    'very_active_minutes': 'sum',
    'sedentary_minutes': 'sum'
}

# Apply aggregation
df_daily = df_daily.groupby('date').agg(agg_rules).reset_index()

# Check result
df_daily.head()

,date,calories,distance,steps,lightly_active_minutes,moderately_active_minutes,very_active_minutes,sedentary_minutes
0,2025-05-04,1425.60,0.0,0.0,0.0,0.0,0.0,1440.0
1,2025-05-05,1813.57,398600.0,3869.0,138.0,2.0,12.0,1288.0
2,2025-05-06,1778.81,175551.0,1864.0,161.0,0.0,0.0,1279.0
3,2025-05-07,1828.60,401850.0,4352.0,169.0,5.0,8.0,1258.0
4,2025-05-08,1909.28,319459.0,3477.0,239.0,5.0,2.0,1194.0


In [8]:
df_daily

,date,calories,distance,steps,lightly_active_minutes,moderately_active_minutes,very_active_minutes,sedentary_minutes
0,2025-05-04,1425.60,0.0,0.0,0.0,0.0,0.0,1440.0
1,2025-05-05,1813.57,398600.0,3869.0,138.0,2.0,12.0,1288.0
2,2025-05-06,1778.81,175551.0,1864.0,161.0,0.0,0.0,1279.0
3,2025-05-07,1828.60,401850.0,4352.0,169.0,5.0,8.0,1258.0
4,2025-05-08,1909.28,319459.0,3477.0,239.0,5.0,2.0,1194.0
...,...,...,...,...,...,...,...,...
239,2025-12-29,1751.28,186662.0,1979.0,129.0,6.0,14.0,1291.0
240,2025-12-30,1766.06,312437.0,3270.0,164.0,1.0,6.0,1269.0
241,2025-12-31,1813.65,555257.0,5977.0,128.0,15.0,12.0,1285.0
242,2026-01-01,1708.99,152043.0,1585.0,110.0,0.0,0.0,1330.0


In [9]:
# Save as parquet
df_daily.to_parquet("C:/Users/bisar/My fitbit dashboard project/Daily Data/daily_aggregated.parquet", index=False)

# Optional: save as CSV too
df_daily.to_csv("C:/Users/bisar/My fitbit dashboard project/Daily Data/daily_aggregated.csv", index=False)
